In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from sklearn import preprocessing
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction import FeatureHasher

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path = '/Users/charlie/Desktop/insurance/'
#glob.glob(os.path.join(path,'*'))

In [3]:
#tr = pd.read_csv(path+ 'training-set.csv')
#te = pd.read_csv(path+ 'testing-set.csv')
#claim = pd.read_csv(path+ 'claim_0702.csv')
#policy = pd.read_csv(path+ 'policy_0702.csv')
tr = pd.read_csv(path+ 'tr3.csv')
te = pd.read_csv(path+ 'te3.csv')
#print len(tr)
#print len(te)

In [19]:
policy["no_coverage_deductible"] = np.where(policy.Coverage_Deductible_if_applied == 0, 1, 0)
policy["withdrawl_policy"] = np.where(policy.Coverage_Deductible_if_applied < 0, 1, 0)

In [20]:
tmp = policy.groupby('Policy_Number', as_index=False)['no_coverage_deductible'].sum()
tmp.columns = ['Policy_Number', 'no_coverage_deductible_count']
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [21]:
tmp = policy.groupby('Policy_Number', as_index=False)['withdrawl_policy'].sum()
tmp.columns = ['Policy_Number', 'withdrawl_policy_count']
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [ ]:
policy.loc[policy.fmarriage.isnull(),'fmarriage'] = '0' 
policy.loc[policy.fmarriage == ' ', 'fmarriage'] = '0'
policy.fmarriage = policy.fmarriage.astype('int')

In [73]:
df = policy.groupby('Policy_Number')[["Policy_Number","Insured's_ID","Prior_Policy_Number","Cancellation",
                                     "Vehicle_identifier","fmarriage","ibirth","fassured"]].first()

In [74]:
tmp = pd.merge(df, claim, on=['Policy_Number'])

In [30]:
# 是否曾汽車過戶
tmp2 = tmp[tmp["Driver's_Relationship_with_Insured"] == 1]
tmp2['same_driver'] = np.where(tmp2.ibirth == tmp2.DOB_of_Driver, 1, 0)
print tmp2.same_driver.value_counts()
tmp3 = tmp2.groupby('Policy_Number', as_index=0)["same_driver"].min()
tmp3["transfer"] = np.where(tmp3.same_driver == 1, 0, 1)
tr = pd.merge(tr, tmp3[["Policy_Number","transfer"]], how = 'left', on = 'Policy_Number')
te = pd.merge(te, tmp3[["Policy_Number","transfer"]], how = 'left', on = 'Policy_Number') 

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
# 是否曾換婚姻狀態
tmp2 = tmp[tmp["Driver's_Relationship_with_Insured"] == 1]
tmp2['same_fmarriage'] = np.where(tmp2.fmarriage == tmp2.Marital_Status_of_Driver, 1, 0)
print tmp2.same_fmarriage.value_counts()
tmp3 = tmp2.groupby('Policy_Number', as_index=0)["same_fmarriage"].min()
tmp3["divorce_marriage"] = np.where(tmp3.same_fmarriage == 1, 0, 1)
tr = pd.merge(tr, tmp3[["Policy_Number","divorce_marriage"]], how = 'left', on = 'Policy_Number')
te = pd.merge(te, tmp3[["Policy_Number","divorce_marriage"]], how = 'left', on = 'Policy_Number') 

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1    29617
0      517
Name: same_fmarriage, dtype: int64


In [86]:
# 是否曾換車牌
tmp2 = tmp[tmp["Driver's_Relationship_with_Insured"] == 1]
tmp2['same_Vehicle'] = np.where(tmp2.Vehicle_identifier_x == tmp2.Vehicle_identifier_y, 1, 0)
#print tmp2.same_Vehicle.value_counts()
tmp3 = tmp2.groupby('Policy_Number', as_index=0)["same_Vehicle"].min()
tmp3["transfer_vehicle"] = np.where(tmp3.same_Vehicle == 1, 0, 1)
tr = pd.merge(tr, tmp3[["Policy_Number","transfer_vehicle"]], how = 'left', on = 'Policy_Number')
te = pd.merge(te, tmp3[["Policy_Number","transfer_vehicle"]], how = 'left', on = 'Policy_Number') 

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
tr.to_csv(path+'tr3.csv', index=False, encoding='utf-8')
te.to_csv(path+'te3.csv', index=False, encoding='utf-8')

In [18]:
tmp = claim.groupby(['Claim_Number','Policy_Number'],as_index=0)["number_of_claimants"].first()
tmp = tmp.groupby('Policy_Number', as_index=0).agg({"number_of_claimants":["sum",'mean']})
tmp.columns = ["Policy_Number",'sum_number_of_claimants','mean_number_of_claimants']
tr = pd.merge(tr, tmp, how = 'left', on = 'Policy_Number')
te = pd.merge(te, tmp, how = 'left', on = 'Policy_Number') 